# HU Extension              Assignment 08      E63 Big Data Analytics 		                  	
## Handed out: 03/24/2023                         Due by 6:00 PM EST on Saturday, 04/01/2023
## Emmanuel Aboah

### Problem 1. 
Install Neo4j on your local machine or your Ubuntu VM. You may also install it on a fresh AMI in Amazon Web Services (the AMI should not have Noe4j already installed).  Show the commands you used to install Neo4j.  Change permission in the neo4j config file to disable authentication. Start Neo4j server and demonstrate that you can login into Neo4j browser and it displays a message that authentication is disabled. Create a new database called YourShortFirstName.  Show the names of databases that you can access.  Show if there are any nodes attached to your new database.

For my installation I will be using an offical neo4j docker image via docker.

```yaml
version: '3.3'
services:
  neo4j:
    image: neo4j:latest
    container_name: neo4j
    ports:
      - 7474:7474
      - 7687:7687
    volumes:
      - neo4j:/var/lib/neo4j
    environment:
      - NEO4J_AUTH=neo4j/password
volumes:
  neo4j:
```

Run the container via docker-compose

```bash
docker-compose up -d
[+] Running 1/1
 ⠿ Container neo4j  Started
```

```bash
# Update docker conf to disable auth 
vi conf/neo4j.conf

# Require (or disable the requirement of) auth to access Neo4j
dbms.security.auth_enabled=false

# Restart the container
(BDA2023) manny@LAPTOP-85L1BUVJ:~/dev/cscie-63/hw08/docker$ docker-compose restart neo4j
[+] Running 1/1
 ⠿ Container neo4j  Started  
```

Login to the UI app with no credentials.

![login portal](./images/neoLogin1.png)

Dasboard

![success](./images/neo_dashboard.png)

neo4j-shell

```bash
# Show databases:
@neo4j> show databases;
+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| name     | type       | aliases | access       | address          | role      | writer | requestedStatus | currentStatus | statusMessage | default | home  | constituents |
+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| "neo4j"  | "standard" | []      | "read-write" | "localhost:7687" | "primary" | TRUE   | "online"        | "online"      | ""            | TRUE    | TRUE  | []           |
| "system" | "system"   | []      | "read-write" | "localhost:7687" | "primary" | TRUE   | "online"        | "online"      | ""            | FALSE   | FALSE | []           |
+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------+

2 rows

# Create database
vi conf/neo4j.conf
# The name of the default database
initial.dbms.default_database=manny

# Show databases
@manny> show databases;
+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| name     | type       | aliases | access       | address          | role      | writer | requestedStatus | currentStatus | statusMessage | default | home  | constituents |
+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| "manny"  | "standard" | []      | "read-write" | "localhost:7687" | "primary" | TRUE   | "online"        | "online"      | ""            | TRUE    | TRUE  | []           |
| "neo4j"  | "standard" | []      | "read-write" | "localhost:7687" | "primary" | TRUE   | "offline"       | "unknown"     | ""            | FALSE   | FALSE | []           |
| "system" | "system"   | []      | "read-write" | "localhost:7687" | "primary" | TRUE   | "online"        | "online"      | ""            | FALSE   | FALSE | []           |
+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
```

Show nodes

![image.png](./images/nodes.png)

### Problem 2.
Create 4 csv files for data on slide 97, titled Note on position of csv-files directory.  You can use provided files or you can create your own. Upload these 4 csv files into neo4j using the browser or the cypher-shell and show the statements you used.  Show all nodes and relationships that are created as a graph and as a table display.  Delete the Oliver Stone node.

```bash
# Upload files from local directory to neo4j import directory
(BDA2023) manny@LAPTOP-85L1BUVJ:~/dev/cscie-63/hw08/docker$ docker cp ../files/movies.csv neo4j:/var/lib/neo4j/import
(BDA2023) manny@LAPTOP-85L1BUVJ:~/dev/cscie-63/hw08/docker$ docker cp ../files/movie_actor_role.csv neo4j:/var/lib/neo4j/import
(BDA2023) manny@LAPTOP-85L1BUVJ:~/dev/cscie-63/hw08/docker$ docker cp ../files/roles.csv neo4j:/var/lib/neo4j/import
(BDA2023) manny@LAPTOP-85L1BUVJ:~/dev/cscie-63/hw08/docker$ docker cp ../files/persons.csv neo4j:/var/lib/neo4j/import

# Load persons
@manny> LOAD CSV WITH HEADERS FROM "file:///persons.csv" AS line
        MERGE (a:Person {id:line.id })
        ON CREATE SET a.name=line.name;
0 rows
ready to start consuming query after 819 ms, results consumed after another 0 ms
Added 5 nodes, Set 10 properties, Added 5 labels

# Load movies
@manny> LOAD CSV WITH HEADERS FROM "file:///movies.csv" AS line
        CREATE (m:Movie { id:line.id, title:line.title, released:toInteger(line.year) });
0 rows
ready to start consuming query after 126 ms, results consumed after another 0 ms
Added 3 nodes, Set 9 properties, Added 3 labels

# Load roles
@manny> LOAD CSV WITH HEADERS FROM "file:///roles.csv" AS line
        MATCH (m:Movie { id:line.movieId })
        MATCH (a:Person { id:line.personId })
        CREATE (a)-[:ACTED_IN { roles: [line.role]}]->(m);
0 rows
ready to start consuming query after 171 ms, results consumed after another 0 ms
Created 6 relationships, Set 6 properties

# Load movie_actor_roles
@manny> LOAD CSV WITH HEADERS FROM "file:///movie_actor_role.csv" AS line
        FIELDTERMINATOR ";"
        MERGE (m:Movie { title:line.title })
        ON CREATE SET m.released = toInteger(line.released)
        MERGE (a:Person { name:line.actor })
        ON CREATE SET a.born = toInteger(line.born)
        MERGE (a)-[:ACTED_IN { roles:split(line.characters,",")}]->(m);
0 rows
ready to start consuming query after 189 ms, results consumed after another 0 ms
Added 3 nodes, Created 2 relationships, Set 8 properties, Added 3 labels
```

Show all nodes and relationships that are created as a graph and as a table display.

### Graph
![nodes](./images/all_nodes.png)

### Table

```json
{
  "identity": 0,
  "labels": [
    "Person"
  ],
  "properties": {
    "name": "Charlie Sheen",
    "id": "1"
  },
  "elementId": "0"
}
{
  "identity": 1,
  "labels": [
    "Person"
  ],
  "properties": {
    "name": "Oliver Stone",
    "id": "2"
  },
  "elementId": "1"
}
{
  "identity": 2,
  "labels": [
    "Person"
  ],
  "properties": {
    "name": "Michael Douglas",
    "id": "3"
  },
  "elementId": "2"
}
{
  "identity": 3,
  "labels": [
    "Person"
  ],
  "properties": {
    "name": "Martin Sheen",
    "id": "4"
  },
  "elementId": "3"
}
{
  "identity": 4,
  "labels": [
    "Person"
  ],
  "properties": {
    "name": "Morgan Freeman",
    "id": "5"
  },
  "elementId": "4"
}
{
  "identity": 5,
  "labels": [
    "Movie"
  ],
  "properties": {
    "id": "1",
    "title": "Wall Street",
    "released": 1987
  },
  "elementId": "5"
}
{
  "identity": 6,
  "labels": [
    "Movie"
  ],
  "properties": {
    "id": "2",
    "title": "The American President",
    "released": 1995
  },
  "elementId": "6"
}
{
  "identity": 7,
  "labels": [
    "Movie"
  ],
  "properties": {
    "id": "3",
    "title": "The Shawshank Redemption",
    "released": 1994
  },
  "elementId": "7"
}
{
  "identity": 8,
  "labels": [
    "Movie"
  ],
  "properties": {
    "title": "Back to the Future",
    "released": 1985
  },
  "elementId": "8"
}
{
  "identity": 9,
  "labels": [
    "Person"
  ],
  "properties": {
    "born": 1961,
    "name": "Michael J. Fox"
  },
  "elementId": "9"
}
{
  "identity": 10,
  "labels": [
    "Person"
  ],
  "properties": {
    "born": 1938,
    "name": "Christopher Lloyd"
  },
  "elementId": "10"
}
```

### TEXT

```text
╒═════════════════════════════════════════════════════════════╕
│"n"                                                          │
╞═════════════════════════════════════════════════════════════╡
│{"name":"Charlie Sheen","id":"1"}                            │
├─────────────────────────────────────────────────────────────┤
│{"name":"Oliver Stone","id":"2"}                             │
├─────────────────────────────────────────────────────────────┤
│{"name":"Michael Douglas","id":"3"}                          │
├─────────────────────────────────────────────────────────────┤
│{"name":"Martin Sheen","id":"4"}                             │
├─────────────────────────────────────────────────────────────┤
│{"name":"Morgan Freeman","id":"5"}                           │
├─────────────────────────────────────────────────────────────┤
│{"id":"1","title":"Wall Street","released":1987}             │
├─────────────────────────────────────────────────────────────┤
│{"id":"2","title":"The American President","released":1995}  │
├─────────────────────────────────────────────────────────────┤
│{"id":"3","title":"The Shawshank Redemption","released":1994}│
├─────────────────────────────────────────────────────────────┤
│{"title":"Back to the Future","released":1985}               │
├─────────────────────────────────────────────────────────────┤
│{"born":1961,"name":"Michael J. Fox"}                        │
├─────────────────────────────────────────────────────────────┤
│{"born":1938,"name":"Christopher Lloyd"}                     │
└─────────────────────────────────────────────────────────────┘
```

Delete Node "Oliver Twist".

```bash
match (n:Person {name: "Oliver Stone"}) detach delete n;

Deleted 1 node, completed after 14 ms.

# Show remaining records
@manny> MATCH (n) RETURN n LIMIT 25;
+-----------------------------------------------------------------------+
| n                                                                     |
+-----------------------------------------------------------------------+
| (:Person {name: "Charlie Sheen", id: "1"})                            |
| (:Person {name: "Michael Douglas", id: "3"})                          |
| (:Person {name: "Martin Sheen", id: "4"})                             |
| (:Person {name: "Morgan Freeman", id: "5"})                           |
| (:Movie {id: "1", title: "Wall Street", released: 1987})              |
| (:Movie {id: "2", title: "The American President", released: 1995})   |
| (:Movie {id: "3", title: "The Shawshank Redemption", released: 1994}) |
| (:Movie {title: "Back to the Future", released: 1985})                |
| (:Person {name: "Michael J. Fox", born: 1961})                        |
| (:Person {name: "Christopher Lloyd", born: 1938})                     |
+-----------------------------------------------------------------------+
```

### Problem 3.
Demonstrate use of the Neo4J Python driver illustrated on slides 114 (neo4j) and beyond.  First run the code to create and find nodes as a script in Python.  Show your results for finding nodes.  Also display the nodes and relationships in your browser.  Delete all older nodes before running your script.  Create a query where you show Alice’s friends. Delete all nodes and relationships when done. 

```bash
# Delete all nodes and ralationships
@manny> MATCH (n) DETACH DELETE n;
0 rows
ready to start consuming query after 132 ms, results consumed after another 0 ms
Deleted 10 nodes, Deleted 8 relationships

# Show empty database
@manny> MATCH (n) RETURN n LIMIT 25;
+---+
| n |
+---+
+---+

0 rows
ready to start consuming query after 1 ms, results consumed after another 0 ms
```

In [3]:
"""🐍 __python__ script to connect to neo4j for creates and deletes. Per problem 3
"""
from neo4j import GraphDatabase, ManagedTransaction

# connection uri
uri = "neo4j://localhost:7687"

driver = GraphDatabase.driver(uri=uri)

def create_person(tx: ManagedTransaction, person):
    tx.run("CREATE (a:Person {name: $name})", name = person)

def create_friend_of(tx, name, friend):
    tx.run("MATCH (a:Person) WHERE a.name = $name "
           "CREATE (a)-[:KNOWS]->(:Person {name: $friend})",
           name=name, friend=friend)

with driver.session() as session:
    session.execute_write(create_person, "Alice")

with driver.session() as session:
    session.execute_write(create_friend_of, "Alice", "Bob")

with driver.session() as session:
    session.execute_write(create_friend_of, "Alice", "Carl")

driver.close()

Query for records added by python script.

```bash
@manny> MATCH (n) RETURN n LIMIT 25;
+---------------------------+
| n                         |
+---------------------------+
| (:Person {name: "Bob"})   |
| (:Person {name: "Alice"}) |
| (:Person {name: "Carl"})  |
+---------------------------+

3 rows
ready to start consuming query after 1 ms, results consumed after another 1 ms
```

Nodes showing in browser.

![alice's friends](./images/friend_of.png)

Query to show Alice's friends.

```bash
@manny> match (p:Person {name: "Alice"})-[r:KNOWS]->(f:Person) return p, r, f;
+-----------------------------------------------------------------+
| p                         | r        | f                        |
+-----------------------------------------------------------------+
| (:Person {name: "Alice"}) | [:KNOWS] | (:Person {name: "Bob"})  |
| (:Person {name: "Alice"}) | [:KNOWS] | (:Person {name: "Carl"}) |
+-----------------------------------------------------------------+
```

Delete all records.

```bash
@manny> MATCH (n) DETACH DELETE n;
0 rows
ready to start consuming query after 4 ms, results consumed after another 0 ms
Deleted 3 nodes, Deleted 2 relationships
@manny> MATCH (n) RETURN n LIMIT 25;
+---+
| n |
+---+
+---+

0 rows
ready to start consuming query after 1 ms, results consumed after another 1 ms
```

### Problem 4.
Create a new database called movies on your Neo4j server. Load attached file create_movies_db.cypher using Cypher-Shell. While using Cypher Shell tell use how many relationships of each kind you have.

```bash
# Load data into database
(BDA2023) manny@LAPTOP-85L1BUVJ:~/dev/cscie-63$ docker exec -it neo4j cypher-shell -f import/create_movies_db.cypher
+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| a                                         | m                                                                                                                                                                               | d                                                    |
+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| (:Person {name: "Tom Hanks", born: 1956}) | (:Movie {tagline: "In every life there comes a time when that thing you dream becomes that thing you do", title: "That Thing You Do", released: 1996})                          | (:Person {name: "Tom Hanks", born: 1956})            |
| (:Person {name: "Tom Hanks", born: 1956}) | (:Movie {tagline: "Once in a lifetime you get a chance to do something different.", title: "A League of Their Own", released: 1992})                                            | (:Person {name: "Penny Marshall", born: 1943})       |
| (:Person {name: "Tom Hanks", born: 1956}) | (:Movie {tagline: "What if someone you never met, someone you never saw, someone you never knew was the only someone for you?", title: "Sleepless in Seattle", released: 1993}) | (:Person {name: "Nora Ephron", born: 1941})          |
| (:Person {name: "Tom Hanks", born: 1956}) | (:Movie {tagline: "A stiff drink. A little mascara. A lot of nerve. Who said they couldn't bring down the Soviet empire.", title: "Charlie Wilson's War", released: 2007})      | (:Person {name: "Mike Nichols", born: 1931})         |
| (:Person {name: "Tom Hanks", born: 1956}) | (:Movie {tagline: "At the edge of the world, his journey begins.", title: "Cast Away", released: 2000})                                                                         | (:Person {name: "Robert Zemeckis", born: 1951})      |
| (:Person {name: "Tom Hanks", born: 1956}) | (:Movie {tagline: "Walk a mile you'll never forget.", title: "The Green Mile", released: 1999})                                                                                 | (:Person {name: "Frank Darabont", born: 1959})       |
| (:Person {name: "Tom Hanks", born: 1956}) | (:Movie {tagline: "Break The Codes", title: "The Da Vinci Code", released: 2006})                                                                                               | (:Person {name: "Ron Howard", born: 1954})           |
| (:Person {name: "Tom Hanks", born: 1956}) | (:Movie {tagline: "This Holiday Season... Believe", title: "The Polar Express", released: 2004})                                                                                | (:Person {name: "Robert Zemeckis", born: 1951})      |
| (:Person {name: "Tom Hanks", born: 1956}) | (:Movie {tagline: "A story of love, lava and burning desire.", title: "Joe Versus the Volcano", released: 1990})                                                                | (:Person {name: "John Patrick Stanley", born: 1950}) |
| (:Person {name: "Tom Hanks", born: 1956}) | (:Movie {tagline: "Everything is connected", title: "Cloud Atlas", released: 2012})                                                                                             | (:Person {name: "Tom Tykwer", born: 1965})           |
+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+

10 rows
ready to start consuming query after 2843 ms, results consumed after another 284 ms
Added 171 nodes, Created 253 relationships, Set 564 properties, Added 171 labels
```

Number of relationship types.

```bash
@manny> MATCH (n)-[r]->(p) RETURN count(distinct type(r)) as relationship_types_count;
+--------------------------+
| relationship_types_count |
+--------------------------+
| 6                        |
+--------------------------+

1 row
ready to start consuming query after 26 ms, results consumed after another 1 ms
@manny> MATCH (n)-[r]->(p) RETURN distinct type(r) as relationship_types;
+--------------------+
| relationship_types |
+--------------------+
| "PRODUCED"         |
| "ACTED_IN"         |
| "DIRECTED"         |
| "WROTE"            |
| "FOLLOWS"          |
| "REVIEWED"         |
+--------------------+

6 rows
ready to start consuming query after 38 ms, results consumed after another 1 ms
```

# Problem 5.
Choose a node with several relationships by visually inspecting your graph in Neo4j browser.  Use the following query to confirm the number of relationships that particular node participates in:

```bash
MATCH (n:MyNodeLabel)
WHERE n.property = "value"
RETURN n, size((n)-->()) AS count
```

Now, for nodes of Movie Label select the number of relationships for each node and report those in the descending order. Limit report to the top 10 nodes. 

```bash
# Count the number of relationships the Movie "The Matrix" has
@manny> match (m:Movie {title: "The Matrix"})-[r]-(:Person)
        return count(r) as matrix_counts;
+---------------+
| matrix_counts |
+---------------+
| 8             |
+---------------+

# Top 10 Movies with the the highest degree of relationships.
@manny> match (m:Movie)-[r]-(p:Person)
        return m.title, count(r) as num_of_rel
        order by num_of_rel desc
        limit 10;
+-------------------------------------+
| m.title                | num_of_rel |
+-------------------------------------+
| "A Few Good Men"       | 14         |
| "Jerry Maguire"        | 13         |
| "Speed Racer"          | 12         |
| "V for Vendetta"       | 11         |
| "Cloud Atlas"          | 10         |
| "The Green Mile"       | 9          |
| "Top Gun"              | 8          |
| "The Matrix"           | 8          |
| "When Harry Met Sally" | 8          |
| "Stand By Me"          | 8          |
+-------------------------------------+

10 rows
ready to start consuming query after 56 ms, results consumed after another 2 ms
```

# Problem 6.
Keanu Reeves acted in the movie “John Wick 3” which is not in the databases we built in Problem 1. Find the director(s) of the movie and add them as attributes. Cast of the movie included Laurence Fishburne and Halle Berry. Add all those people (persons) and the roles they played in this movie to the database using a single Cypher command. Do not forget to add John Wicks dog, its bread and name. Do that using a Python Client. Using a Cypher command in the browser generate a graph showing all the movies in which Keanu Reeves played a role and all other actors that also played in those movies. Generate a textual report using Cypher-shell. 

```bash
# Create John wick movie and relationships
@manny>CREATE (JohnWick3:Movie {title:'John Wick 3', released:2019, tagline:'Parabellum'})
CREATE (Chad:Person {name:'Chad Stahelski', born:1968})
CREATE
(Chad)-[:DIRECTED]->(JohnWick3);
#second query
MATCH (Keanu:Person {name: "Keanu Reeves"})
MATCH (Laurence:Person {name: "Laurence Fishburne"})
MATCH (HalleB:Person {name: "Halle Berry"})
MATCH (JohnWick3:Movie {title: "John Wick 3"})
CREATE
(Keanu)-[:ACTED_IN {roles:['John Wick']}]->(JohnWick3),
(Laurence)-[:ACTED_IN {roles:['Bowery King']}]->(JohnWick3),
(HalleB)-[:ACTED_IN {roles:['sofia']}]->(JohnWick3);
0 rows
ready to start consuming query after 1 ms, results consumed after another 0 ms
Added 2 nodes, Created 1 relationships, Set 5 properties, Added 2 labels
0 rows
ready to start consuming query after 107 ms, results consumed after another 0 ms
Created 3 relationships, Set 3 properties
```

In [7]:
"""🐍 __python__ script to add John Wick. Per problem 6
"""
from neo4j import GraphDatabase, ManagedTransaction

# connection uri
uri = "neo4j://localhost:7687"

driver = GraphDatabase.driver(uri=uri)

def create_dog(tx: ManagedTransaction, dog, breed):
    tx.run("CREATE (d:Dog {name: $name, breed: $breed})", name = dog, breed=breed)

def create_acted_in(tx, name, role, movie):
    tx.run("MATCH (d:Dog) WHERE d.name = $name "
           "MATCH (m:Movie) WHERE m.title = $movie "
           "CREATE (d)-[:ACTED_IN {roles:[$role]}]->(:Movie {title: $movie})",
           name=name, role=role, movie=movie)

with driver.session() as session:
    session.execute_write(create_dog, "Andy", "beagle")

with driver.session() as session:
    session.execute_write(create_acted_in, "Andy", "Cha Cha", "John Wick 3")

driver.close()

in the browser generate a graph showing all the movies in which Keanu Reeves played a role and all other actors that also played in those movies.

![johnWick](./images/johnWick.png)

Generate a textual report using Cypher-shell.
```bash
@manny> Match (p:Person {name: "Keanu Reeves"})-[r:ACTED_IN]->(m:Movie)<-[o:ACTED_IN]-(a) where (a:Person or a:Dog) return p,r,m,o,a;
+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| p                                            | r                                        | m                                                                                                                      | o                                          | a                                                  |
+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| (:Person {name: "Keanu Reeves", born: 1964}) | [:ACTED_IN {roles: ["John Wick"]}]       | (:Movie {tagline: "Parabellum", title: "John Wick 3", released: 2019})                                                 | [:ACTED_IN {roles: ["sofia"]}]             | (:Person {name: "Halle Berry", born: 1966})        |
| (:Person {name: "Keanu Reeves", born: 1964}) | [:ACTED_IN {roles: ["John Wick"]}]       | (:Movie {tagline: "Parabellum", title: "John Wick 3", released: 2019})                                                 | [:ACTED_IN {roles: ["Bowery King"]}]       | (:Person {name: "Laurence Fishburne", born: 1961}) |
| (:Person {name: "Keanu Reeves", born: 1964}) | [:ACTED_IN {roles: ["Julian Mercer"]}]   | (:Movie {title: "Something's Gotta Give", released: 2003})                                                             | [:ACTED_IN {roles: ["Erica Barry"]}]       | (:Person {name: "Diane Keaton", born: 1946})       |
| (:Person {name: "Keanu Reeves", born: 1964}) | [:ACTED_IN {roles: ["Julian Mercer"]}]   | (:Movie {title: "Something's Gotta Give", released: 2003})                                                             | [:ACTED_IN {roles: ["Harry Sanborn"]}]     | (:Person {name: "Jack Nicholson", born: 1937})     |
| (:Person {name: "Keanu Reeves", born: 1964}) | [:ACTED_IN {roles: ["Shane Falco"]}]     | (:Movie {tagline: "Pain heals, Chicks dig scars... Glory lasts forever", title: "The Replacements", released: 2000})   | [:ACTED_IN {roles: ["Annabelle Farrell"]}] | (:Person {name: "Brooke Langton", born: 1970})     |
| (:Person {name: "Keanu Reeves", born: 1964}) | [:ACTED_IN {roles: ["Shane Falco"]}]     | (:Movie {tagline: "Pain heals, Chicks dig scars... Glory lasts forever", title: "The Replacements", released: 2000})   | [:ACTED_IN {roles: ["Clifford Franklin"]}] | (:Person {name: "Orlando Jones", born: 1968})      |
| (:Person {name: "Keanu Reeves", born: 1964}) | [:ACTED_IN {roles: ["Shane Falco"]}]     | (:Movie {tagline: "Pain heals, Chicks dig scars... Glory lasts forever", title: "The Replacements", released: 2000})   | [:ACTED_IN {roles: ["Jimmy McGinty"]}]     | (:Person {name: "Gene Hackman", born: 1930})       |
| (:Person {name: "Keanu Reeves", born: 1964}) | [:ACTED_IN {roles: ["Johnny Mnemonic"]}] | (:Movie {tagline: "The hottest data on earth. In the coolest head in town", title: "Johnny Mnemonic", released: 1995}) | [:ACTED_IN {roles: ["J-Bone"]}]            | (:Person {name: "Ice-T", born: 1958})              |
| (:Person {name: "Keanu Reeves", born: 1964}) | [:ACTED_IN {roles: ["Johnny Mnemonic"]}] | (:Movie {tagline: "The hottest data on earth. In the coolest head in town", title: "Johnny Mnemonic", released: 1995}) | [:ACTED_IN {roles: ["Jane"]}]              | (:Person {name: "Dina Meyer", born: 1968})         |
| (:Person {name: "Keanu Reeves", born: 1964}) | [:ACTED_IN {roles: ["Johnny Mnemonic"]}] | (:Movie {tagline: "The hottest data on earth. In the coolest head in town", title: "Johnny Mnemonic", released: 1995}) | [:ACTED_IN {roles: ["Takahashi"]}]         | (:Person {name: "Takeshi Kitano", born: 1947})     |
| (:Person {name: "Keanu Reeves", born: 1964}) | [:ACTED_IN {roles: ["Kevin Lomax"]}]     | (:Movie {tagline: "Evil has its winning ways", title: "The Devil's Advocate", released: 1997})                         | [:ACTED_IN {roles: ["John Milton"]}]       | (:Person {name: "Al Pacino", born: 1940})          |
| (:Person {name: "Keanu Reeves", born: 1964}) | [:ACTED_IN {roles: ["Kevin Lomax"]}]     | (:Movie {tagline: "Evil has its winning ways", title: "The Devil's Advocate", released: 1997})                         | [:ACTED_IN {roles: ["Mary Ann Lomax"]}]    | (:Person {name: "Charlize Theron", born: 1975})    |
| (:Person {name: "Keanu Reeves", born: 1964}) | [:ACTED_IN {roles: ["Neo"]}]             | (:Movie {tagline: "Everything that has a beginning has an end", title: "The Matrix Revolutions", released: 2003})      | [:ACTED_IN {roles: ["Agent Smith"]}]       | (:Person {name: "Hugo Weaving", born: 1960})       |
| (:Person {name: "Keanu Reeves", born: 1964}) | [:ACTED_IN {roles: ["Neo"]}]             | (:Movie {tagline: "Everything that has a beginning has an end", title: "The Matrix Revolutions", released: 2003})      | [:ACTED_IN {roles: ["Morpheus"]}]          | (:Person {name: "Laurence Fishburne", born: 1961}) |
| (:Person {name: "Keanu Reeves", born: 1964}) | [:ACTED_IN {roles: ["Neo"]}]             | (:Movie {tagline: "Everything that has a beginning has an end", title: "The Matrix Revolutions", released: 2003})      | [:ACTED_IN {roles: ["Trinity"]}]           | (:Person {name: "Carrie-Anne Moss", born: 1967})   |
| (:Person {name: "Keanu Reeves", born: 1964}) | [:ACTED_IN {roles: ["Neo"]}]             | (:Movie {tagline: "Free your mind", title: "The Matrix Reloaded", released: 2003})                                     | [:ACTED_IN {roles: ["Agent Smith"]}]       | (:Person {name: "Hugo Weaving", born: 1960})       |
| (:Person {name: "Keanu Reeves", born: 1964}) | [:ACTED_IN {roles: ["Neo"]}]             | (:Movie {tagline: "Free your mind", title: "The Matrix Reloaded", released: 2003})                                     | [:ACTED_IN {roles: ["Morpheus"]}]          | (:Person {name: "Laurence Fishburne", born: 1961}) |
| (:Person {name: "Keanu Reeves", born: 1964}) | [:ACTED_IN {roles: ["Neo"]}]             | (:Movie {tagline: "Free your mind", title: "The Matrix Reloaded", released: 2003})                                     | [:ACTED_IN {roles: ["Trinity"]}]           | (:Person {name: "Carrie-Anne Moss", born: 1967})   |
| (:Person {name: "Keanu Reeves", born: 1964}) | [:ACTED_IN {roles: ["Neo"]}]             | (:Movie {tagline: "Welcome to the Real World", title: "The Matrix", released: 1999})                                   | [:ACTED_IN {roles: ["Agent Smith"]}]       | (:Person {name: "Hugo Weaving", born: 1960})       |
| (:Person {name: "Keanu Reeves", born: 1964}) | [:ACTED_IN {roles: ["Neo"]}]             | (:Movie {tagline: "Welcome to the Real World", title: "The Matrix", released: 1999})                                   | [:ACTED_IN {roles: ["Morpheus"]}]          | (:Person {name: "Laurence Fishburne", born: 1961}) |
| (:Person {name: "Keanu Reeves", born: 1964}) | [:ACTED_IN {roles: ["Neo"]}]             | (:Movie {tagline: "Welcome to the Real World", title: "The Matrix", released: 1999})                                   | [:ACTED_IN {roles: ["Trinity"]}]           | (:Person {name: "Carrie-Anne Moss", born: 1967})   |
| (:Person {name: "Keanu Reeves", born: 1964}) | [:ACTED_IN {roles: ["Neo"]}]             | (:Movie {tagline: "Welcome to the Real World", title: "The Matrix", released: 1999})                                   | [:ACTED_IN {roles: ["Emil"]}]              | (:Person {name: "Emil Eifrem", born: 1978})        |
+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+

22 rows
ready to start consuming query after 1 ms, results consumed after another 2 ms
```